## Questão 01

In [35]:
import numpy as np
import random
from sklearn.model_selection import train_test_split

In [38]:
response_vectors = np.array([
    [1, -1, -1, -1, -1, -1, -1, -1],
    [-1, 1, -1, -1, -1, -1, -1, -1],
    [-1, -1, 1, -1, -1, -1, -1, -1],
    [-1, -1, -1, 1, -1, -1, -1, -1],
    [-1, -1, -1, -1, 1, -1, -1, -1],
    [-1, -1, -1, -1, -1, 1, -1, -1],
    [-1, -1, -1, -1, -1, -1, 1, -1],
    [-1, -1, -1, -1, -1, -1, -1, 1]
], dtype=np.float32)

data_X = []
data_y = []

for i in range(10000):
    instance = np.array([random.randint(0,1), random.randint(0,1), random.randint(0,1)])
    label = instance[0]*4 + instance[1]*2 + instance[2]
    noise = np.array([random.uniform(-0.1, 0.1), random.uniform(-0.1, 0.1), random.uniform(-0.1, 0.1)])

    data_X.append(instance+noise)
    data_y.append(label)

data_X = np.array(data_X, dtype=np.float32)
data_y = np.array(data_y, dtype=np.float32)

X_train, X_test, y_train, y_test = train_test_split(data_X, data_y, test_size=0.2, random_state=10)

In [56]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

clf = Perceptron(tol=1e-3, random_state=10, validation_fraction=0.2, early_stopping=True, verbose=True)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(y_test, y_pred))

-- Epoch 1
Norm: 3.23, NNZs: 3, Bias: 1.000000, T: 6400, Avg. loss: 0.000353
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 3.23, NNZs: 3, Bias: 1.000000, T: 12800, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 3.23, NNZs: 3, Bias: 1.000000, T: 19200, Avg. loss: 0.000000
Total training time: 0.01 seconds.
-- Epoch 4
Norm: 3.23, NNZs: 3, Bias: 1.000000, T: 25600, Avg. loss: 0.000000
Total training time: 0.01 seconds.
-- Epoch 5
Norm: 3.23, NNZs: 3, Bias: 1.000000, T: 32000, Avg. loss: 0.000000
Total training time: 0.01 seconds.
-- Epoch 6
Norm: 3.23, NNZs: 3, Bias: 1.000000, T: 38400, Avg. loss: 0.000000
Total training time: 0.01 seconds.
Convergence after 6 epochs took 0.01 seconds
-- Epoch 1
Norm: 4.42, NNZs: 3, Bias: -1.000000, T: 6400, Avg. loss: 0.001058
Total training time: 0.00 seconds.
-- Epoch 2
Norm: 4.42, NNZs: 3, Bias: -1.000000, T: 12800, Avg. loss: 0.000000
Total training time: 0.00 seconds.
-- Epoch 3
Norm: 4.42, NNZs: 3, Bias: -1.000000, T:

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s finished


In [60]:
## plotar gráfico tridimensional com os pontos e suas respectivas classes usando o plotly
import plotly.graph_objects as go
fig = go.Figure(data=[go.Scatter3d(x=X_test[:,0], y=X_test[:,1], z=X_test[:,2], mode='markers', marker=dict(size=3, color=y_pred))])
fig.show()

## Questão 02

In [73]:
n = 10000
data_X = []
data_y = []

for i in range(n):
    instance = np.array([random.uniform(-5,5), random.uniform(-5,5)])

    if instance[0]*instance[1] > 0:
        label = 1
    elif instance[0]*instance[1] < 0:
        label = -1
    else:
        continue

    data_X.append(instance)
    data_y.append(label)


data_X = np.array(data_X, dtype=np.float32)
data_y = np.array(data_y, dtype=np.float32)

In [78]:
from torchvision import datasets, transform
import torch
import torch.nn as nn


ModuleNotFoundError: No module named 'torchvision'